In [208]:
import os
from pathlib import Path
import cv2
import numpy as np
import kagglehub

path = kagglehub.dataset_download("trainingdatapro/cars-video-object-tracking")
print("Path to dataset files:", path)

DATASET_DIR = Path(path)

Path to dataset files: /Users/Marta/.cache/kagglehub/datasets/trainingdatapro/cars-video-object-tracking/versions/3


In [209]:
IMAGE_DIR = DATASET_DIR / "images"

assert IMAGE_DIR.exists(), f"Missing {IMAGE_DIR}"
def sorted_images(folder: Path):
    exts = (".png", ".jpg", ".jpeg", ".bmp")
    files = [p for p in folder.iterdir() if p.suffix.lower() in exts]
    files.sort(key=lambda p: p.name)
    return files

IMAGE_FILES = sorted_images(IMAGE_DIR)

print("Frames:", len(IMAGE_FILES))


Frames: 301


## Detection

This section extracts moving objects from video frames using background subtraction and morphological cleaning. We use the MOG2 algorithm to separate foreground (moving objects) from the static background. To remove noise, we apply morphological operations such as opening, closing, and dilation. We then use connected components with perspective awareness to extract individual objects from the mask. 

Because the mask is imperfect due to various factors like vehicle color, distance to camera, and lighting conditions, a single car sometimes appears divided into multiple disconnected blobs. That's why we cluster and group the blobs that are likely to be part of the same vehicle, taking perspective into account. Since blobs far from the camera that are close together could belong to different vehicles, but as they come closer to the camera the distance between vehicles increases, we use perspective-aware clustering to  merge fragments while avoiding over-merging distant vehicles.

### Functions

TODO: explain each function

In [210]:
def clean_mask(mask: np.ndarray) -> np.ndarray:

    # Remove shadow pixels from MOG2 
    _, mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)
    
    k2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k2, iterations=2)
    return mask


def cluster_blobs_by_proximity(dets: list, distance_threshold: float =100.0) -> list:
    """
    Merge blobs that are very close to each other (likely fragments of same vehicle).
    Simplified: assumes cars are well-defined, uses fixed distance threshold.
    """
    if len(dets) <= 1:
        return dets
    
    n = len(dets)
    clusters = list(range(n))
    
    def find_root(i):
        if clusters[i] != i:
            clusters[i] = find_root(clusters[i])
        return clusters[i]
    
    def union(i, j):
        root_i, root_j = find_root(i), find_root(j)
        if root_i != root_j:
            clusters[root_j] = root_i
    
    # Simple pairwise distance check
    for i in range(n):
        for j in range(i + 1, n):
            cx_i, cy_i = dets[i]["centroid"]
            cx_j, cy_j = dets[j]["centroid"]
            d = ((cx_j - cx_i)**2 + (cy_j - cy_i)**2) ** 0.5
            
            if d < distance_threshold:
                union(i, j)
    
    # Merge clusters
    cluster_map = {}
    for i in range(n):
        root = find_root(i)
        if root not in cluster_map:
            cluster_map[root] = []
        cluster_map[root].append(i)
    
    merged = []
    for cluster_indices in cluster_map.values():
        cluster_dets = [dets[i] for i in cluster_indices]
        if len(cluster_dets) == 1:
            merged.append(cluster_dets[0])
        else:
            total_area = sum(d["area"] for d in cluster_dets)
            merged_cx = sum(d["centroid"][0] * d["area"] for d in cluster_dets) / total_area
            merged_cy = sum(d["centroid"][1] * d["area"] for d in cluster_dets) / total_area
            
            all_xs = [d["bbox"][0] for d in cluster_dets] + [d["bbox"][0] + d["bbox"][2] for d in cluster_dets]
            all_ys = [d["bbox"][1] for d in cluster_dets] + [d["bbox"][1] + d["bbox"][3] for d in cluster_dets]
            x_min, x_max = min(all_xs), max(all_xs)
            y_min, y_max = min(all_ys), max(all_ys)
            
            merged.append({
                "centroid": (int(merged_cx), int(merged_cy)),
                "bbox": (x_min, y_min, x_max - x_min, y_max - y_min),
                "area": total_area
            })
    
    return merged

def allowed_area_range(y, img_h):
    """
    Perspective model with realistic lower bounds for distant vehicles.
    Uses conservative minimum (300 px²) to catch small distant cars,
    while maintaining quadratic growth for closer vehicles.
    """
    t = y / img_h   # 0 top → 1 bottom

    # FIXED: Lower base to catch distant cars (300-800 px²)
    # Original empirical model was too restrictive (base=2000)
    min_area = 300 + 25000 * t * t  # Allows 300 px² at top, ~25k at bottom
    max_area = 800 + 110000 * t * t  # Allows 800 px² at top, ~110k at bottom

    return min_area, max_area

def detect_blobs(mask: np.ndarray, roi_mask=None, min_area=1500):

    if roi_mask is not None:
        mask = cv2.bitwise_and(mask, roi_mask)

    H, W = mask.shape[:2]

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    dets = []
    for c in contours:
        area = cv2.contourArea(c)
        if area <= 0:
            continue

        x, y, w, h = cv2.boundingRect(c)

        # perspective filtering: area must be plausible for its vertical position (y)
        minA, maxA = allowed_area_range(y, H)  # Use y (top) instead of y+h (bottom)
        if not (minA <= area <= maxA):
            continue

        if area < 150:
            continue  # Lower minimum area threshold for small cars

        M = cv2.moments(c)
        if M["m00"] == 0:
            continue

        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])

        dets.append({"centroid": (cx, cy), "bbox": (x, y, w, h), "area": area})

    # Cluster and merge nearby fragmented blobs (perspective-aware)
    dets = cluster_blobs_by_proximity(dets, distance_threshold=40.0)
    return dets


## Tracking

Next, we need to track each detected car. We chose the Kalman filter because it is fast, supports multiple object tracking, and predicts the probable next position of each car based on its own motion model. Different cars move at different speeds, and the Kalman filter learns and adapts to these individual motion patterns.

The Kalman filter maintains a state vector [x, y, vx, vy] representing position and velocity. The transition matrix defines how the state evolves between frames (it predicts where the car will be based on its current velocity). The measurement matrix maps the detected position (which we observe) to the state space. The process noise covariance controls how much we trust the motion model (lower values = trust motion more), while the measurement noise covariance controls how much we trust the detections (lower values = trust detections more). By balancing these, the filter smooths noisy detections while allowing the object to change speed.

Each detection is attached to a Track object, which aims to maintain a persistent identity throughout the video. Every track has a unique ID and its own Kalman filter. The predict function estimates where the vehicle should be in the current frame using the motion model, before we see any new detections. The update function corrects the prediction using the actual detected position, allowing the filter to learn and adjust if the vehicle's motion changes.


In [211]:

def init_kalman(dt: float = 1.0,
                process_var: float = 50.0,
                meas_var: float = 25.0) -> cv2.KalmanFilter:
    """
    State: [x, y, vx, vy]^T
    Measurement: [x, y]^T
    """
    kf = cv2.KalmanFilter(4, 2)

    kf.transitionMatrix = np.array([
        [1, 0, dt, 0 ],
        [0, 1, 0 , dt],
        [0, 0, 1 , 0 ],
        [0, 0, 0 , 1 ],
    ], dtype=np.float32)

    kf.measurementMatrix = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
    ], dtype=np.float32)

    # These are the two main tuning knobs.
    kf.processNoiseCov = np.eye(4, dtype=np.float32) * process_var
    kf.measurementNoiseCov = np.eye(2, dtype=np.float32) * meas_var

    # Start uncertain so it can adapt quickly.
    kf.errorCovPost = np.eye(4, dtype=np.float32) * 500.0
    kf.statePost = np.zeros((4, 1), dtype=np.float32)
    return kf

def color_from_id(track_id: int) -> tuple[int, int, int]:

    rng = np.random.default_rng(track_id)  # stable seed per id
    # Keep colors away from extremes: [40..220]
    c = rng.integers(40, 220, size=3, dtype=np.int32)
    return (int(c[0]), int(c[1]), int(c[2]))  # B, G, R

class Track:
    """
    One vehicle hypothesis + identity.
    We keep:
      - Kalman filter
      - hits: how many times we matched a detection (confidence)
      - missed: how many consecutive frames we failed to match (death timer)
      - confirmed: whether track has been stable for MIN_HITS frames
    """
    def __init__(self, track_id: int, init_xy: tuple[int,int], init_bbox, dt=1.0,
                 process_var=50.0, meas_var=25.0):
        self.id = track_id
        self.kf = init_kalman(dt=dt, process_var=process_var, meas_var=meas_var)
        self.color = color_from_id(track_id)

        x, y = init_xy
        self.kf.statePost = np.array([[x], [y], [0], [0]], dtype=np.float32)

        self.hits = 1
        self.missed = 0
        self.confirmed = False  # Only show after MIN_HITS consecutive matches
        self.bbox = init_bbox
        self.history = [init_xy]
        self.consecutive_hits = 1  # Count consecutive frames with detections

    def predict(self) -> tuple[float,float]:
        """
        Predict where the vehicle should be in the current frame (before seeing detections).
        Important: use the predicted output from predict(), not statePost.
        """
        pred = self.kf.predict()
        return float(pred[0]), float(pred[1])

    def update(self, xy: tuple[int,int], bbox):
        """
        Correct the predicted state using the detection measurement.
        """
        cx, cy = xy
        z = np.array([[cx], [cy]], dtype=np.float32)
        self.kf.correct(z)
        self.hits += 1
        self.consecutive_hits += 1  # Increment consecutive hits
        self.missed = 0
        self.bbox = bbox
        self.history.append(xy)

    def mark_missed(self):
        """
        No detection matched this track this frame.
        """
        self.missed += 1
        self.consecutive_hits = 0  # Reset consecutive hits

    def is_confirmed(self, min_hits: int = 3) -> bool:
        """Check if track is stable and should be displayed."""
        return self.consecutive_hits >= min_hits


In [212]:
# adaptive association with measurement validation

def match_distance_for_bbox(bbox):
    """
    Larger objects can move more pixels between frames.
    bbox height ≈ distance to camera (perspective effect).
    Adaptive gate: closer objects (larger) can move more.
    """
    _, _, w, h = bbox
    return 20 + 0.8 * h   # slightly more permissive than before

def euclidean_distance(track, detection):
    """
    Compute Euclidean distance between track prediction and detection.
    """
    pred_state = track.kf.statePost
    pred_x, pred_y = float(pred_state[0]), float(pred_state[1])
    det_x, det_y = detection["centroid"]
    d = ((pred_x - det_x)**2 + (pred_y - det_y)**2) ** 0.5
    return d


def associate_detections_to_tracks(dets, tracks, min_confirmed_hits=3):
    """
    For each track, pick the nearest detection,
    but only if it lies within a size-dependent distance.
    
    Prioritize confirmed tracks to prevent ghost IDs from stealing detections.
    Validate measurements: reject detections that would cause large corrections
    to avoid mask artifacts corrupting track state.
    
    OPTIMIZED: Use sets instead of list.remove() to avoid O(n) operations.
    """

    matches = []
    unmatched_dets = set(range(len(dets)))
    unmatched_tracks = set(range(len(tracks)))

    # First pass: match CONFIRMED tracks (higher priority)
    for ti, t in enumerate(tracks):
        if not t.is_confirmed(min_confirmed_hits) or not unmatched_dets:
            continue

        tx, ty = t.history[-1]
        best_di = None
        best_d = float("inf")

        for di in unmatched_dets:
            cx, cy = dets[di]["centroid"]
            d = ((tx - cx)**2 + (ty - cy)**2) ** 0.5
            allowed = match_distance_for_bbox(dets[di]["bbox"])

            if d < allowed and d < best_d:
                best_d = d
                best_di = di

        if best_di is not None:
            # Additional validation: check that this detection is plausible
            # using Kalman filter's predicted state (not just history)
            euclid_d = euclidean_distance(t, dets[best_di])
            pred_allowed = match_distance_for_bbox(dets[best_di]["bbox"]) * 1.2  # 20% relaxation for KF
            if euclid_d < pred_allowed:
                matches.append((ti, best_di))
                unmatched_dets.discard(best_di)
                unmatched_tracks.discard(ti)

    # Second pass: match UNCONFIRMED tracks (lower priority, stricter matching)
    for ti, t in enumerate(tracks):
        if t.is_confirmed(min_confirmed_hits) or ti not in unmatched_tracks or not unmatched_dets:
            continue

        tx, ty = t.history[-1]
        best_di = None
        best_d = float("inf")

        for di in unmatched_dets:
            cx, cy = dets[di]["centroid"]
            d = ((tx - cx)**2 + (ty - cy)**2) ** 0.5
            # Stricter gate for unconfirmed tracks (avoid spurious matches)
            allowed = match_distance_for_bbox(dets[di]["bbox"]) * 0.7

            if d < allowed and d < best_d:
                best_d = d
                best_di = di

        if best_di is not None:
            matches.append((ti, best_di))
            unmatched_dets.discard(best_di)
            unmatched_tracks.discard(ti)

    return matches, list(unmatched_tracks), list(unmatched_dets)


In [ ]:
# main multi-object tracking loop with ID fragmentation fixes

SAVE_VIDEO = True
VIDEO_NAME = "vehicle_tracking_debug.mp4"
VIDEO_FPS = 20
video_writer = None

DT = 1.0
MAX_DIST = 50.0      # association gate (pixels)
MAX_AGE  = 15        # INCREASED: frames allowed to miss before deleting (allows recovery from mask gaps)
MIN_HITS = 3         # show track after this many consecutive matches
MIN_AREA = 1200      # blob area threshold (slightly lower to catch fragments)

PROCESS_VAR = 1e-2   # OpenCV KF units
MEAS_VAR    = 1e-1

# Blob merging parameters
MERGE_DISTANCE = 50.0  # pixels: merge blobs closer than this
SIZE_RATIO = 0.6       # allow merging if area ratio is between 0.6 and 1/0.6

# Background subtractor (learns background online)
bg = cv2.createBackgroundSubtractorMOG2(history=400, varThreshold=16, detectShadows=True)

# ROI mask: set to None or supply a binary mask (255=keep, 0=ignore)
roi_mask = None

tracks = []
next_id = 1

SHOW_EVERY = 1   # increase if too slow
SCALE = 0.6      # scale for display screen

def draw_bbox(img, bbox, label, color, thickness=2):
    x, y, w, h = bbox
    x2, y2 = x + w, y + h

    # optional "shadow" outline for contrast
    cv2.rectangle(img, (x, y), (x2, y2), (0, 0, 0), thickness + 2, lineType=cv2.LINE_AA)
    cv2.rectangle(img, (x, y), (x2, y2), color, thickness, lineType=cv2.LINE_AA)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.55
    text_thickness = 2
    (tw, th), baseline = cv2.getTextSize(label, font, font_scale, text_thickness)

    y_text_top = y - (th + baseline + 6)
    if y_text_top < 0:
        y_text_top = y + 2

    x_text = x
    y_text = y_text_top + th + 3

    cv2.rectangle(img, (x_text, y_text_top), (x_text + tw + 8, y_text_top + th + baseline + 6),
                  (0, 0, 0), -1, lineType=cv2.LINE_AA)
    cv2.rectangle(img, (x_text + 1, y_text_top + 1), (x_text + tw + 7, y_text_top + th + baseline + 5),
                  color, -1, lineType=cv2.LINE_AA)

    cv2.putText(img, label, (x_text + 4, y_text),
                font, font_scale, (0, 0, 0), text_thickness, lineType=cv2.LINE_AA)


# Main loop 
# Each car (track) gets its own Kalman filter, which maintains a state vector for position and velocity.
#For every frame, the Kalman filter predicts where the car should be (the predicted position).
# When new detections (blobs) are found, the code tries to match them to existing tracks using both the last known position and the Kalman filter's prediction.
#If a detection matches a track, the Kalman filter updates (corrects) its state using the detected position.


for i, img_path in enumerate(IMAGE_FILES):
    frame = cv2.imread(str(img_path))
    if frame is None:
        continue

    H, W = frame.shape[:2]

    # 1) Foreground mask (motion)
    fg = bg.apply(frame)

    # 2) Clean the mask
    fg_clean = clean_mask(fg)

    # 3) Detections = blobs (with merging of fragments)
    dets = detect_blobs(fg_clean, roi_mask=roi_mask, min_area=MIN_AREA)

    # 4) Predict all tracks (KF motion model)
    pred_xy = [t.predict() for t in tracks]

    # 5) Associate with two-pass approach (confirmed tracks first)
    matches, unmatched_tracks, unmatched_dets = associate_detections_to_tracks(
        dets, tracks, min_confirmed_hits=MIN_HITS
    )

    # 6) Update matched tracks (KF correct step)
    for ti, di in matches:
        cx, cy = dets[di]["centroid"]
        bbox = dets[di]["bbox"]
        tracks[ti].update((cx, cy), bbox)

    # 7) Mark unmatched tracks
    for ti in unmatched_tracks:
        tracks[ti].mark_missed()

    # 8) Create new tracks for unmatched detections
    for di in unmatched_dets:
        cx, cy = dets[di]["centroid"]
        bbox = dets[di]["bbox"]

        # duplicate suppression: don't create new ID if near any existing track 
        duplicate = False
        for t in tracks:
            tx, ty = t.history[-1]
            dist = ((tx - cx)**2 + (ty - cy)**2) ** 0.5
            threshold = 40 if not t.is_confirmed(MIN_HITS) else 50
            if dist < threshold:
                duplicate = True
                break
        if duplicate:
            continue

        # Only create track for reasonably sized detections (avoid noise)
        if dets[di]["area"] < MIN_AREA * 0.8:
            continue

        # Allow new track creation anywhere in the frame

        tracks.append(Track(
            track_id=next_id,
            init_xy=(cx, cy),
            init_bbox=bbox,
            dt=DT,
            process_var=PROCESS_VAR,
            meas_var=MEAS_VAR
        ))
        next_id += 1

    # 9) Delete only truly stale tracks (high MAX_AGE tolerance)
    tracks = [t for t in tracks if t.missed <= MAX_AGE]

    # 10) Visualize + record (mask top, tracking bottom)
    if i % SHOW_EVERY == 0:
        vis = frame.copy()

        # Draw only CONFIRMED tracks (prevents ID flicker)
        for t in tracks:
            if not t.is_confirmed(MIN_HITS):
                continue
            if t.missed > 1:
                continue
            label = f"vehicle_{t.id}"
            draw_bbox(vis, t.bbox, label, t.color, thickness=2)

        mask_vis = cv2.cvtColor(fg_clean, cv2.COLOR_GRAY2BGR)

        vis_small  = cv2.resize(vis, (int(W * SCALE), int(H * SCALE)))
        mask_small = cv2.resize(mask_vis, (int(W * SCALE), int(H * SCALE)))

        stacked = np.vstack([mask_small, vis_small])

        # init writer once
        if SAVE_VIDEO and video_writer is None:
            hh, ww = stacked.shape[:2]
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
            video_writer = cv2.VideoWriter(VIDEO_NAME, fourcc, VIDEO_FPS, (ww, hh))
            print("Recording video to:", VIDEO_NAME)

        if SAVE_VIDEO and video_writer is not None:
            video_writer.write(stacked)

        cv2.imshow("vehicle tracking", stacked)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Cleanup
if video_writer is not None:
    video_writer.release()
    print("Saved:", VIDEO_NAME)

cv2.destroyAllWindows()



Recording video to: vehicle_tracking_debug.mp4


/var/folders/sz/4qbyglh16n3gzj4l7_jkj3rh0000gn/T/ipykernel_8084/800847209.py:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(pred[0]), float(pred[1])
/var/folders/sz/4qbyglh16n3gzj4l7_jkj3rh0000gn/T/ipykernel_8084/1739340089.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = float(pred_state[0]), float(pred_state[1])
/var/folders/sz/4qbyglh16n3gzj4l7_jkj3rh0000gn/T/ipykernel_8084/1739340089.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated Num

Saved: vehicle_tracking_debug.mp4
